## Aplicación de POO: PACMAN

En esta guía vamos a empezar a diseñar una versión básica del tradicional juego PACMAN, organizando el código a partir de algunos conceptos de POO.

HABLAR DE LAS VENTAJAS DE POO?

Qué elementos van a ser clases? Pensemos en cómo es el juego...
En primer lugar, existe un juego, que surje de la interacción entre el PACMAN, los FANTASMAS, el MAPA... Antes de pensar cómo modelar el juego pensemos en el mapa.

¿Cómo describirían un mapa?

Una versión simple de un mapa podría considerar:
- Ser como una matriz, lo cual lo podríamos hacer con una lista de listas
- Ser accesible en sus posiciones, ya que vamos a querer posicionar a los fantasmas, a pacman y posiblemente otros objetos
- Ser printeable/printable
- Poder preguntarle si una posición existe en el mapa, o qué hay en una determinada posición

In [1]:
class EmptyCell:
    def __repr__(self):
        return " . "

In [2]:
class Map:
    def __init__(self, n_rows, n_cols):
        # primero guardo la cantidad de filas y la cantidad de columnas
        self.n_rows = n_rows
        self.n_cols = n_cols
        # luego, genero un mapa vacío. Nota: ésto se puede hacer mejor usando setters, dejémoslo para otra oportunidad
        self._map = [[EmptyCell() for i in range(self.n_cols)] for x in range(self.n_rows)]
    
    def __getitem__(self, row):
        # esta es una función mágica, nos va a permitir acceder de una forma muy pythonezca
        return self._map[row] # pueden adivinar qué nos va a devolver?
    
    def __str__(self):
        # esta es otra función mágica, nos va a dar un print muy canchero, y luego la vamos a modificar más
        str_map = ""
        for row in self._map:
            for col_idx in range(len(row)):
                str_map += str(row[col_idx])
                if col_idx == len(row) - 1:
                    str_map += "\n"
                
        return str_map

Primero, veamos para qué sirve...

In [3]:
# instanciamos un mapa de 5 filas por 3 columnas
game_map = Map(5,3)

In [4]:
game_map[4]

[ . ,  . ,  . ]

In [5]:
print(game_map)

 .  .  . 
 .  .  . 
 .  .  . 
 .  .  . 
 .  .  . 



¿Qué va a pasar a continuación...?

In [6]:
game_map[4][4]

IndexError: list index out of range

¿Tienen algo en común un fantasma y el pacman? Podríamos decir que, al menos, tienen en común que...

1- Tienen una posición en el mapa

2- Se pueden mover

3- Pueden ser comidos (cuando pacman come la pastilla puede comer a los fantasmas)

 Ya que tanto los fantasmas como PACMAN pueden hacer ciertas cosas en común, podemos resumir ese comportamiento en una clase más "abstracta". Es decir, tanto los fantasmas como PACMAN tienen ciertos atributos y acciones en común, en ambos casos son "personajes" del juego. Por este motivo, comenzamos creando esa clase.

In [7]:
import random

In [8]:
class PowerPill:
    def __repr__(self):
        return " O "

In [9]:
class Wall:
    def __repr__(self):
        return " | "

In [10]:
class Ghost:
    def __repr__(self):
        return " G "

In [11]:
class Pacman:
    def __repr__(self):
        return " P "

In [12]:
class Map:
    def __init__(self, n_rows:int, n_cols:int):
        # primero guardo la cantidad de filas y la cantidad de columnas
        self.n_rows = n_rows
        self.n_cols = n_cols
        # luego, genero un mapa vacío. Nota: ésto se puede hacer mejor usando setters, dejémoslo para otra oportunidad
        self._elements = [[EmptyCell() for i in range(self.n_cols)] for x in range(self.n_rows)]
    
    def __getitem__(self, row:int):
        # esta es una función mágica, nos va a permitir acceder de una forma muy pythonezca
        return self._elements[row] # pueden adivinar qué nos va a devolver?
    
    def __str__(self):
        str_map = ""
        for row in self._elements:
            for col_idx in range(len(row)):
                element = row[col_idx]
                str_map += str(element)
                if col_idx == len(row) - 1:
                    str_map += "\n"
                
        return str_map

    def update_element(self, element, pos_x, pos_y):
        # método para modificar el tablero
        self._elements[pos_x][pos_y] = element

    def get_element(self, x, y):
      # método para tomar un elemento
      return self._elements[x][y]

In [13]:
game_map = Map(5,3)

In [14]:
print(game_map)

 .  .  . 
 .  .  . 
 .  .  . 
 .  .  . 
 .  .  . 



In [15]:
game_map.update_element(Pacman(),2,2)

In [16]:
print(game_map)

 .  .  . 
 .  .  . 
 .  .  P 
 .  .  . 
 .  .  . 



In [ ]:
import sys

class Game:
    
    def __init__(self):
        # cuando instanciamos el juego creamos personajes. Por ahora las posiciones son fijas pero se podrían hacer al azar
        initial_map = Map(7,8)
        initial_map.update_element(Pacman(),4,3)
        initial_map.update_element(Ghost(),1,3)
        initial_map.update_element(Ghost(),0,0)
        self.game_map = initial_map
        self.score = 0

    def start(self):
        end = False
        while not end:
            print(f"score:{self.score}")
            print(self.game_map)
            self._random_move_ghosts()
            move_input = input()
            if not self._input_key_is_valid(move_input):
                print("El input tiene que ser a, w, s o d")
            else:
                status = self._update_map_return_status(move_input)
            if status == 1:
                print("Cuidado con la pared")
            if status == 2:
                print("Perdiste")
                end = True
            elif status == 0:
                self.score += 1

    def _get_ghosts_positions(self):
        ghosts_positions = []
        for row in range(self.game_map.n_rows):
            for col in range(self.game_map.n_cols):
                element = self.game_map.get_element(row,col)
                if isinstance(element,Ghost):
                    ghosts_positions.append((row,col))
        return ghosts_positions
    
    
    def _random_move_ghosts(self):
        ghosts_positions = self._get_ghosts_positions()
        for position in ghosts_positions:
            new_x, new_y = self._new_ghost_position(position)
            if isinstance(self.game_map.get_element(new_x, new_y), Pacman):
                print("Perdiste!")
            else:
                self.game_map.update_element(Ghost(),new_x, new_y)
                self.game_map.update_element(EmptyCell(),position[0], position[1])
        

    def _new_ghost_position(self,ghosts_position):
        possible_moves = ["a","w","s","d"]
        random.shuffle(possible_moves)
        for move in possible_moves:
            new_x, new_y = self._advance_position(ghosts_position[0], ghosts_position[1], move)
            if not self._hits_wall(new_x, new_y) and \
                not isinstance(self.game_map.get_element(new_x, new_y), Ghost):
                return new_x, new_y

    def _input_key_is_valid(self, move_input):
        if move_input in ["a","w","s","d"]:
            return True
        else:
            return False

    def _get_pacman_position(self):
        for row in range(self.game_map.n_rows):
            for col in range(self.game_map.n_cols):
            element = self.game_map.get_element(row,col)
            if isinstance(element,Pacman):
                return row, col


    def _update_map_return_status(self, move_input):
        pacman_x,pacman_y = self._get_pacman_position()
        new_x, new_y = self._advance_position(pacman_x,pacman_y,move_input)
        if self._hits_wall(new_x, new_y):
            return 1
        elif isinstance(self.game_map.get_element(new_x,new_y),Ghost): 
            return 2
        else:
            self.game_map.update_element(EmptyCell(), pacman_x, pacman_y)
            self.game_map.update_element(Pacman(), new_x, new_y)
        return 0

    def _hits_wall(self, x, y):
        if x >= 0 and x + 1 < self.game_map.n_cols and y >= 0 and y + 1 < self.game_map.n_rows:
            return False
        return True


    @staticmethod
    def _advance_position(x,y,move):
        if move == "a":
            y = y - 1
        elif move == "w":
            x = x - 1
        elif move == "s":
            x = x + 1
        elif move == "d":
            y = y + 1
        else:
            raise NotImplementedError(f"{move_input}")
        return x,y


In [ ]:
game = Game()

In [ ]:
game.start()

In [ ]:
ss
